<a href="https://colab.research.google.com/github/ekaterinatao/NER_biomed_domain/blob/main/transformers_base/%D0%92%D0%9A%D0%A0_nerel_bio_ruBERT_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Инструменты
Предобработанный дасасет [NEREL-BIO](https://huggingface.co/datasets/ekaterinatao/nerel_bio_ner_unnested)  
[Чек-пойнт](https://huggingface.co/ekaterinatao/nerel-bio-rubert-base) дообученной модели на всем датасете nerel-bio  

Исходная модель [RuBERT](https://huggingface.co/DeepPavlov/rubert-base-cased)

### Установка зависимостей

In [ ]:
!pip install datasets accelerate evaluate wandb seqeval -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import random
from dataclasses import dataclass

import torch
import datasets
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification
from transformers import TrainingArguments, Trainer
from transformers import pipeline
import evaluate

import warnings
warnings.filterwarnings("ignore")

In [ ]:
@dataclass
class TrainingConfig:
    seed = 64
    dataset = 'ekaterinatao/nerel_bio_ner_unnested'
    checkpoint = 'DeepPavlov/rubert-base-cased'
    n_labels = 45
    n_epochs = 10
    train_batch_size = 6
    eval_batch_size = 6
    device = "cuda" if torch.cuda.is_available() else "cpu"
    l_rate = 5e-05
    w_decay = 0.1
    warm_up = 0.1

config = TrainingConfig()

In [ ]:
seed = config.seed

random.seed(seed)
np.random.seed(seed)

torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

### Скачивание датасета

In [ ]:
dataset = datasets.load_dataset(config.dataset)
dataset

Generating train split:   0%|          | 0/612 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/77 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/77 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'words', 'ner_tags'],
        num_rows: 612
    })
    valid: Dataset({
        features: ['id', 'words', 'ner_tags'],
        num_rows: 77
    })
    test: Dataset({
        features: ['id', 'words', 'ner_tags'],
        num_rows: 77
    })
})

In [ ]:
for token, ner_tag in zip(dataset['train'][0]['words'], dataset['train'][0]['ner_tags']):
    print(f'{token:_<60}{ner_tag}')

161_________________________________________________________28
ОКБ_________________________________________________________30
12-275 мес__________________________________________________20
95,65±8,4 мес_______________________________________________20
23__________________________________________________________28
14,29%______________________________________________________31
8___________________________________________________________28
23,13 мес___________________________________________________20
11 мес______________________________________________________20
Более 36 мес________________________________________________20
142_________________________________________________________28
104,87 мес__________________________________________________20
15__________________________________________________________28
80,27 мес___________________________________________________20
3___________________________________________________________28
13,45%_________________________________________________

In [ ]:
# Labels
url = 'https://raw.githubusercontent.com/ekaterinatao/NER_biomed_domain/main/labels.txt'
tags = pd.read_csv(url, names=['tag']).values.tolist()
tags = [item for sublist in tags for item in sublist]
tag_to_id = {tag: i for i, tag in enumerate(tags)}
id_to_tag = {i: tag for i, tag in enumerate(tags)}

___
### Токенизация

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(config.checkpoin)

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
example = dataset["train"][64]
tokenized_input = tokenizer(example["words"], truncation=True, max_length=512, is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])

print(f'Token len initial tokens__{len(example["words"])}\n')
print(f'Token len from BERT tokenizer__{len(tokens)}')
print(' '.join(tokens))

Token len initial tokens__77

Token len from BERT tokenizer__189
[CLS] И ##БС тремя ишем ##ической болезни сердца 301 1066 США 1517 США транс ##жи ##р ##ных кислот пище ишем ##ической болезни сердца пищевых калорий Т ##ЖК жир ##ы масла Т ##ЖК Т ##ЖК И ##БС лип ##идов плазме лип ##идам биом ##арке ##рам воспал ##ительного процесса лип ##идов смерть Т ##ЖК смертей Т ##ЖК населения ответственных лиц населения Т ##ЖК Аргентине сердца до 2004 года 1 , 5 % Т ##ЖК пищевых снижения уровня Т ##ЖК И ##БС процентной доли снижения И ##БС И ##БС сердца И ##БС 523 ##7 лет ежегодных 17 млн долл . США пищевых Т ##ЖК пищевых проспект ##ивных кого ##рт ##ных исследованиях проспект ##ивных кого ##рт ##ных исследований ежегодно 537 ##3 И ##БС 26 394 года острых случаев И ##БС острых случая И ##БС 87 млн долл . США Аргентине болезни сердца калорий уровень потребления риски возникновения И ##БС популя ##ционной выборки профиля ##м лип ##идов болезни сердца профил ##е лип ##идов предотвратить спасти 523 ##7 

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["words"], truncation=True, max_length=512, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_idxs = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_idxs:
            if word_idx is None:
                label_ids.append(-100) # Set the special tokens to -100.
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)
tokenized_dataset

Map:   0%|          | 0/612 [00:00<?, ? examples/s]

Map:   0%|          | 0/77 [00:00<?, ? examples/s]

Map:   0%|          | 0/77 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'words', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 612
    })
    valid: Dataset({
        features: ['id', 'words', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 77
    })
    test: Dataset({
        features: ['id', 'words', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 77
    })
})

In [ ]:
df = tokenized_dataset['train']
pd.DataFrame({
    'words': df['words'], 'ner_tags': df['ner_tags'],
    'input_ids': df['input_ids'], 'labels': df['labels']
}).head(5)

,words,ner_tags,input_ids,labels
0,"[161, ОКБ, 12-275 мес, 95,65±8,4 мес, 23, 14,2...","[28, 30, 20, 20, 28, 31, 28, 20, 20, 20, 28, 2...","[101, 20132, 39736, 4367, 130, 24408, 57175, 1...","[-100, 28, 30, 20, -100, -100, -100, 20, -100,..."
1,"[ИМО, 201, от 2 мес до 18 лет, до 74%, 5,5%, И...","[5, 28, 17, 31, 31, 5, 5, 2, 5, 32, 32, 5, 32,...","[101, 61991, 806, 2777, 1641, 140, 57175, 2785...","[-100, 5, -100, 28, 17, -100, -100, -100, -100..."
2,"[ОФТГ, 2,5-3,0 кг, ВПГ, 1-го, Панавир, 6 инсти...","[5, 28, 12, 29, 3, 28, 20, 5, 12, 5, 5, 15, 15...","[101, 96813, 814, 784, 140, 128, 146, 130, 142...","[-100, 5, -100, -100, 28, -100, -100, -100, -1..."
3,"[29, целиакией, 8, ИГ, целиакией, 7, 18, АГД, ...","[28, 5, 28, 3, 5, 28, 28, 7, 28, 28, 28, 28, 5...","[101, 7688, 16934, 87843, 152, 24481, 16934, 8...","[-100, 28, 5, -100, 28, 3, 5, -100, 28, 28, 7,..."
4,"[ОА, 248, РА, от 38 до 65 лет, II, четыре, 1-я...","[5, 28, 5, 17, 29, 28, 29, 29, 29, 29, 3, 29, ...","[101, 805, 778, 25401, 34662, 1641, 11683, 278...","[-100, 5, -100, 28, 5, 17, -100, -100, -100, -..."


In [ ]:
tokenized_dataset = tokenized_dataset.map(remove_columns = ['id', 'words', 'ner_tags'])
tokenized_dataset

Map:   0%|          | 0/612 [00:00<?, ? examples/s]

Map:   0%|          | 0/77 [00:00<?, ? examples/s]

Map:   0%|          | 0/77 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 612
    })
    valid: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 77
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 77
    })
})

### Обучение модели с различными гиперпараметрами (`learning rate`, `decay weight`)

In [ ]:
seqeval = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [tags[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [tags[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
print(f'device is {config.device}')

device is cuda


____
# Подбор гиперпараметров

In [ ]:
LR_VALUES = (1e-5, 2e-5, 5e-5)
DECAY_VALUES = (1e-4, 0.1)
WARM_UP_VALUES = (0, 0.1)

In [ ]:
for i, LR in enumerate(LR_VALUES):
    for j, WD in enumerate(DECAY_VALUES):
        for y, WU in enumerate(WARM_UP_VALUES):

            model = AutoModelForTokenClassification.from_pretrained(
                config.checkpoint, num_labels=config.n_labels,
                id2label=id_to_tag, label2id=tag_to_id
            )
            print(f'Log: training for l_r:{LR}, w_d:{WD}, w_up:{WU}...')

            training_args = TrainingArguments(
                output_dir="token_class_model",
                num_train_epochs=config.n_epochs,
                learning_rate=LR,
                weight_decay=WD,
                warmup_ratio=WU,
                per_device_train_batch_size=config.train_batch_size,
                per_device_eval_batch_size=config.eval_batch_size,
                group_by_length=True,
                optim="adamw_torch",
                lr_scheduler_type="cosine",
                evaluation_strategy="epoch",
                seed = config.seed,
                data_seed = config.seed,
                push_to_hub=False,
                save_strategy="no"
            )

            trainer = Trainer(
                model=model,
                args=training_args,
                train_dataset=tokenized_dataset["train"],
                eval_dataset=tokenized_dataset["valid"],
                tokenizer=tokenizer,
                data_collator=data_collator,
                compute_metrics=compute_metrics,
            )

            trainer.train()

Some weights of BertForTokenClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Log: training for l_r:1e-05, w_d:0.0001, w_up:0...


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,1.497717,0.520128,0.457552,0.486837,0.627972
2,No log,1.058169,0.643254,0.597092,0.619314,0.727428
3,No log,0.841241,0.697225,0.659944,0.678072,0.778089
4,No log,0.750172,0.715015,0.690197,0.702387,0.797669
5,1.063100,0.704931,0.729736,0.707317,0.718352,0.805439
6,1.063100,0.666475,0.736410,0.714822,0.725455,0.812587
7,1.063100,0.654234,0.732614,0.716463,0.724449,0.812898
8,1.063100,0.646552,0.739518,0.719747,0.729498,0.816628
9,1.063100,0.647828,0.738232,0.720919,0.729473,0.815540
10,0.420600,0.647565,0.738015,0.722092,0.729967,0.815385


Some weights of BertForTokenClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Log: training for l_r:1e-05, w_d:0.0001, w_up:0.1...


...

Log: training for l_r:5e-05, w_d:0.1, w_up:0...


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.805948,0.683666,0.666510,0.674979,0.779176
2,No log,0.575234,0.765000,0.753518,0.759216,0.842113
3,No log,0.544066,0.766209,0.759381,0.762780,0.843357
4,No log,0.584620,0.766047,0.764071,0.765058,0.843667
5,0.531200,0.600345,0.771388,0.767589,0.769484,0.850505
6,0.531200,0.603919,0.777856,0.774390,0.776119,0.849728
7,0.531200,0.618860,0.778590,0.774390,0.776484,0.850971
8,0.531200,0.628763,0.782283,0.782833,0.782558,0.853147
9,0.531200,0.631612,0.784471,0.781895,0.783181,0.853924
10,0.080500,0.635425,0.782895,0.781426,0.782160,0.853613


Some weights of BertForTokenClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Log: training for l_r:5e-05, w_d:0.1, w_up:0.1...


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,1.149250,0.614612,0.574109,0.593670,0.708003
2,No log,0.661019,0.727294,0.715525,0.721362,0.813675
3,No log,0.575303,0.758038,0.757505,0.757771,0.838384
4,No log,0.562447,0.773137,0.764071,0.768577,0.845688
5,0.795100,0.621142,0.766604,0.768762,0.767681,0.841026
6,0.795100,0.625854,0.777517,0.776970,0.777243,0.848485
7,0.795100,0.619768,0.783714,0.780957,0.782333,0.851593
8,0.795100,0.626221,0.783032,0.792214,0.787596,0.853147
9,0.795100,0.634473,0.785398,0.789634,0.787510,0.855633
10,0.093900,0.637206,0.787328,0.792683,0.789996,0.856410


___
# Обучение с подобранными гиперпараметрами
Лучшими оказались гиперпараметры: `learning_rate`= 5e-5, `weight_decay` = 0.1, `warm_up` = 0.1  

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import os
os.environ["WANDB_PROJECT"]="ner_bert_nerel_bio"
hf_repo_id = "ekaterinatao/nerel-bio-rubert-base"

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    config.checkpoint, num_labels=config.n_labels, id2label=id_to_tag, label2id=tag_to_id)

training_args = TrainingArguments(
    output_dir=hf_repo_id,
    num_train_epochs=config.n_epochs,
    learning_rate=config.l_rate,
    weight_decay=config.w_decay,
    warmup_ratio=config.warm_up,
    per_device_train_batch_size=config.train_batch_size,
    per_device_eval_batch_size=config.eval_batch_size,
    group_by_length=True,
    optim="adamw_torch",
    lr_scheduler_type="cosine",
    evaluation_strategy="epoch",
    seed = config.seed,
    data_seed = config.seed,
    push_to_hub=True,
    save_strategy="no",
    report_to="wandb",
    run_name="rubert-base",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["valid"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


wandb: Currently logged in as: taoea. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,1.121077,0.619560,0.580910,0.599613,0.712510
2,No log,0.679956,0.733317,0.716463,0.724792,0.813675
3,No log,0.598507,0.744463,0.748827,0.746639,0.830303
4,No log,0.567296,0.760768,0.762195,0.761481,0.840249
5,0.795400,0.566465,0.775077,0.770169,0.772615,0.848485
6,0.795400,0.593402,0.782598,0.774156,0.778354,0.854390
7,0.795400,0.580425,0.779481,0.775094,0.777281,0.852681
8,0.795400,0.607486,0.783897,0.787758,0.785823,0.857653
9,0.795400,0.613945,0.788746,0.788931,0.788838,0.861383
10,0.102400,0.612167,0.787304,0.788227,0.787765,0.860140


TrainOutput(global_step=1020, training_loss=0.4412419216305602, metrics={'train_runtime': 139.794, 'train_samples_per_second': 43.779, 'train_steps_per_second': 7.296, 'total_flos': 856255185792720.0, 'train_loss': 0.4412419216305602, 'epoch': 10.0})

In [ ]:
wandb.finish()

eval/accuracy,▁▆▇▇▇█████
eval/f1,▁▆▆▇▇█████
eval/loss,█▂▁▁▁▁▁▂▂▂
eval/precision,▁▆▆▇▇█████
eval/recall,▁▆▇▇▇█████
eval/runtime,█▄▆▃▅▁▃▅▁▁
eval/samples_per_second,▁▅▃▆▄█▆▄██
eval/steps_per_second,▁▅▃▆▄█▆▄██
train/epoch,▁▂▃▃▄▄▅▆▆▇███
train/global_step,▁▂▃▃▄▄▅▆▆▇███
train/learning_rate,█▁


In [ ]:
trainer.save_model(hf_repo_id)

training_args.bin:   0%|          | 0.00/4.73k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/709M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

### Оценка качества на тестовой выборке

In [ ]:
predictions = trainer.predict(test_dataset=tokenized_dataset["test"])

In [ ]:
predictions.metrics

{'test_loss': 0.6296454071998596,
 'test_precision': 0.7918465227817746,
 'test_recall': 0.7890083632019116,
 'test_f1': 0.7904248952722921,
 'test_accuracy': 0.8646103896103896,
 'test_runtime': 0.8113,
 'test_samples_per_second': 94.915,
 'test_steps_per_second': 16.025}

Оценка модели на абстракте, которого не было в тестовом наборе

In [ ]:
ner_bio = pipeline("ner", model=model, tokenizer=tokenizer, device=config.device)

In [ ]:
abstract = """Цель. Оценить выживаемость у пациентов с болезнью Фабри (БФ) в зависимости от вида заместительной почечной терапии, и определить роль диализного скрининга в ранней диагностике БФ у родственников.
Материалы и методы. В исследование включали взрослых (старше 18 лет) пациентов с подтвержденным диагнозом БФ. Терминальная стадия хронической почечной недостаточности (тХПН) диагностировали в соответствии с рекомендациями Научного общества нефрологов России (2016) и KDIGO (2012). На основании опроса пробандов выявляли его родственников, которые могли унаследовать мутантный ген.
Результаты. У 50 (24,9%) из 201 обследованных пациентов с БФ диагностирована тХПН, в том числе у 48 (40%) из 120 мужчин и 2 (2,7%) из 81 женщин. Оценка кумулятивной частоты методом Каплана-Майера демонстрирует выраженное увеличение частоты регистрации тХПН к возрасту 20-30 лет, а к возрасту 50 лет ожидаемое количество пациентов с тХПН составляет 95%. Пяти из 50 больных с тХПН была выполнена трансплантация почки, в среднем, через 17 месяцев (диапазон от 7 до 70 месяцев) после инициации лечения гемодиализом. Умерло 15 (30%) из 50 пациентов, получавших лечение гемодиализом. Все умершие пациенты были мужского пола. Медиана возраста на момент летального исхода составила 45 (39; 58) лет. Среди пациентов, которым проведена трансплантация почки, летальных исходов зарегистрировано не было. У 44 (88%) из 50 пациентов диагноз БФ установлен, в среднем, через 1 год (диапазон от 0 до 12 лет) после начала лечения программным гемодиализом, в том числе у одного пациента – после трансплантации почки. Среди 44 пробандов, выявленных при всероссийском диализном скрининге, проведен семейный скрининг. Патогенная мутация в гене GLA диагностирована у 89 (57%) из 156 обследованных родственников диализных пробандов, в том числе у 18 детей моложе 18 лет, клинические проявления БФ имелись у 48 родственников. У 80,4% обследованных родственников диализных пробандов обнаружено поражение почек, преимущественно на ранних стадиях.
Заключение. ТХПН нередкое осложнение БФ, ассоциированное с неблагоприятным прогнозом. Однако диализный скрининг –  эффективный способ выявления пробандов с БФ, открывающий возможность установить диагноз БФ у родственников на ранних стадиях, когда лечение наиболее эффективно.
"""

In [ ]:
ner_bio(abstract)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity': 'SCIPROC',
  'score': 0.76758903,
  'index': 1,
  'word': 'Цель',
  'start': 0,
  'end': 4},
 {'entity': 'SCIPROC',
  'score': 0.5501071,
  'index': 2,
  'word': '.',
  'start': 4,
  'end': 5},
 {'entity': 'FINDING',
  'score': 0.8515094,
  'index': 3,
  'word': 'Оцен',
  'start': 6,
  'end': 10},
 {'entity': 'FINDING',
  'score': 0.9006018,
  'index': 4,
  'word': '##ить',
  'start': 10,
  'end': 13},
 {'entity': 'PHYS',
  'score': 0.77701473,
  'index': 5,
  'word': 'выжив',
  'start': 14,
  'end': 19},
 {'entity': 'PHYS',
  'score': 0.7618812,
  'index': 6,
  'word': '##аемость',
  'start': 19,
  'end': 26},
 {'entity': 'PERSON',
  'score': 0.9907691,
  'index': 7,
  'word': 'у',
  'start': 27,
  'end': 28},
 {'entity': 'PERSON',
  'score': 0.9979462,
  'index': 8,
  'word': 'пациентов',
  'start': 29,
  'end': 38},
 {'entity': 'PERSON',
  'score': 0.85923153,
  'index': 9,
  'word': 'с',
  'start': 39,
  'end': 40},
 {'entity': 'DISO',
  'score': 0.99781764,
  'index': 